In [19]:
import torch
import torch.nn as nn

class DGMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(DGMNet, self).__init__()
        self.input_dim = input_dim
        self.hidden1 = nn.Linear(input_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, 1)
        self.activation = nn.Tanh()
        
    def forward(self, t, x):
        inputs = torch.cat([t, x], dim=1)  # Concatenate t and x
        x = self.activation(self.hidden1(inputs))
        x = self.activation(self.hidden2(x))
        return self.output(x)



In [20]:
def compute_loss(model, t, x):
    # Use autograd for derivatives
    u = model(t, x)
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]

    # Compute second-order derivatives w.r.t. each spatial variable
    u_xx = torch.zeros_like(u)
    for i in range(x.size(1)):
        u_xx += torch.autograd.grad(u_x[:, i:i+1].sum(), x, create_graph=True)[0][:, i:i+1]

    # Example PDE residual: ∂u/∂t + Δu - cu = 0
    c = torch.tensor(1.0)
    residual = u_t + u_xx - c * u
    return torch.mean(residual ** 2)


In [21]:
def sample_points(num_samples, T, L):
    t = (torch.rand(num_samples, 1) * T).requires_grad_(True)
    x = torch.cat([torch.rand(num_samples, 1) * l for l in L], dim=1).requires_grad_(True)
    return t, x



In [24]:
T = 1.0  # Time interval upper bound
L = [2, 3]  # Spatial domain bounds for each dimension

model = DGMNet(input_dim=1+len(L), hidden_dim=100)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(501):
    t_samples, x_samples = sample_points(1000, T, L)
    loss = compute_loss(model, t_samples, x_samples)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



Epoch 0, Loss: 0.008074752055108547
Epoch 50, Loss: 7.405990618281066e-05
Epoch 100, Loss: 2.083548861264717e-05
Epoch 150, Loss: 1.3675378795596771e-05
Epoch 200, Loss: 8.473459274682682e-06
Epoch 250, Loss: 7.172058303694939e-06
Epoch 300, Loss: 5.506426532519981e-06
Epoch 350, Loss: 4.468002316571074e-06
Epoch 400, Loss: 4.1500165934849065e-06
Epoch 450, Loss: 2.4412306629528757e-06
Epoch 500, Loss: 2.1241282865958055e-06


In [25]:
# Assume we're evaluating at specific t and across a grid in x
t_eval = torch.tensor([[0.5]])  # Example: Evaluate at t=0.5
x_eval_grid = ...  # Create a grid of x values within the domain

with torch.no_grad():
    u_pred = model(t_eval, x_eval_grid)
    # Visualization or comparison code here


TypeError: expected Tensor as element 1 in argument 0, but got ellipsis